# Quantized Training

Starting from scratch

In [30]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="QTv2-"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v6-QT/"))


print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

# Baseline layer count, and embedding size
L_SIZE=6
D_SIZE=512

# Deepspeed and batch size
DEEPSPEED_STAGE="deepspeed_stage_2"
BATCH_SIZE=4

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v6-QT
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


# Dataset & Model Setup

In [3]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_100k-world-4k-rechunk.yaml"

Map (num_proc=160): 100%|███| 1000000/1000000 [00:37<00:00, 26659.45 examples/s]
Filter (num_proc=160): 100%|█| 1000000/1000000 [00:06<00:00, 149382.73 examples/
Map (num_proc=160): 100%|██████| 124119/124119 [00:20<00:00, 5977.08 examples/s]
Saving the dataset (7/7 shards): 100%|█| 124119/124119 [00:03<00:00, 39297.29 ex
Saving the dataset (1/1 shards): 100%|█| 100/100 [00:00<00:00, 4449.34 examples/


In [11]:
# Baseline layer count, and embedding size
L_SIZE=6
D_SIZE=512

# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer {L_SIZE} --n_embd {D_SIZE} \
        --vocab_size world --skip-if-exists \
        "../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth"
!ls -lh "{TRAINER_DIR}/../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth"

[2024-04-22 05:29:04,801] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-world-v6base-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Output model exists, skipping init_model
-rw-rw-r-- 1 recursal recursal 170M Apr 22 02:46 /home/recursal/RWKV-infctx-trainer/RWKV-v6/../model/L6-D512-world-v6base-init.pth


# L6-D512 , CMix Quantized run

In [17]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE="fp4"
RWKV_TIMX_QTYPE=""

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 02:23:30,410] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
[RWKV] TMIX Quantize type    : 
[RWKV] CMIX Quantize type    : fp4
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D512-world-v6base-init.pth', '--model.lr_init=4e-3', '--model.lr_final=4e-4', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L6-D512-CM_fp4/', '--trainer.logger.init_args.name

In [19]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE="nf4"
RWKV_TIMX_QTYPE=""

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 02:41:28,357] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
[RWKV] TMIX Quantize type    : 
[RWKV] CMIX Quantize type    : nf4
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D512-world-v6base-init.pth', '--model.lr_init=4e-3', '--model.lr_final=4e-4', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L6-D512-CM_nf4/', '--trainer.logger.init_args.name

In [22]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE="4bit"
RWKV_TIMX_QTYPE=""

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 03:45:13,215] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
[RWKV] TMIX Quantize type    : 
[RWKV] CMIX Quantize type    : 4bit
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D512-world-v6base-init.pth', '--model.lr_init=4e-3', '--model.lr_final=4e-4', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L6-D512-CM_4bit/', '--trainer.logger.init_args.na

# L6-D512 , TMix Quantized run

In [31]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE=""
RWKV_TIMX_QTYPE="fp4"

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TM_{RWKV_TIMX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-TM_{RWKV_TIMX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 07:35:38,678] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
[RWKV] TMIX Quantize type    : fp4
[RWKV] CMIX Quantize type    : 
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D512-world-v6base-init.pth', '--model.lr_init=4e-3', '--model.lr_final=4e-4', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L6-D512-TM_fp4/', '--trainer.logger.init_args.name

In [32]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE=""
RWKV_TIMX_QTYPE="nf4"

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TM_{RWKV_TIMX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-TM_{RWKV_TIMX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 07:52:43,760] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
[RWKV] TMIX Quantize type    : nf4
[RWKV] CMIX Quantize type    : 
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D512-world-v6base-init.pth', '--model.lr_init=4e-3', '--model.lr_final=4e-4', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L6-D512-TM_nf4/', '--trainer.logger.init_args.name

In [33]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE=""
RWKV_TIMX_QTYPE="4bit"

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-TM_{RWKV_TIMX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-TM_{RWKV_TIMX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found
[2024-04-26 08:10:05,235] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
[RWKV] TMIX reuse multiplier : 1
[RWKV] CMIX reuse multiplier : 1
[RWKV] TMIX Quantize type    : 4bit
[RWKV] CMIX Quantize type    : 
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/rwkv-x-exp/v6-layerNbits/config/enwiki_100k-world-32k-rechunk.yaml', '--model.load_model=../model/L6-D512-world-v6base-init.pth', '--model.lr_init=4e-3', '--model.lr_final=4e-4', '--trainer.callbacks.init_args.dirpath=../checkpoint/v6-enwiki-100k-L6-D512-TM_4bit/', '--t

[rank: 7] Seed set to 3941088705
initializing deepspeed distributed: GLOBAL_RANK: 7, MEMBER: 8/8
>> Loading dataset from data_path:  ../datapath/enwiki_100k-world-32k-rechunk/
>> Dataset load finished:  ../datapath/enwiki_100k-world-32k-rechunk/
[rank: 2] Seed set to 3941088705
initializing deepspeed distributed: GLOBAL_RANK: 2, MEMBER: 3/8
>> Loading dataset from data_path:  ../datapath/enwiki_100k-world-32k-rechunk/
>> Dataset load finished:  ../datapath/enwiki_100k-world-32k-rechunk/
[rank: 3] Seed set to 3941088705
initializing deepspeed distributed: GLOBAL_RANK: 3, MEMBER: 4/8
>> Loading dataset from data_path:  ../datapath/enwiki_100k-world-32k-rechunk/
>> Dataset load finished:  ../datapath/enwiki_100k-world-32k-rechunk/
[rank: 1] Seed set to 3941088705
initializing deepspeed distributed: GLOBAL_RANK: 1, MEMBER: 2/8
>> Loading dataset from data_path:  ../datapath/enwiki_100k-world-32k-rechunk/
>> Dataset load finished:  ../datapath/enwiki_100k-world-32k-rechunk/
[rank: 5] Seed s

# L6-D512 , CMix & TMix Quantized run

In [34]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE="fp4"
RWKV_TIMX_QTYPE="fp4"

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}-TM_{RWKV_CMIX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}-TM_{RWKV_CMIX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 08:27:20,832] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
Traceback (most recent call last):
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/lightning_trainer.py", line 266, in <module>
    from src.model import RWKV
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/src/model.py", line 7, in <module>
    from .module.CoreDependencies import *
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/src/module/CoreDependencies.py", line 180, in <module>
    RWKV_CMIX_QVARS = os.environ.get("RWKV_CMIX_QVARS", "RKV").to_upper()
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'str' object has no attribute 'to_upper'. Did you mean: 'isupper'?


In [35]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE="nf4"
RWKV_TIMX_QTYPE="nf4"

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}-TM_{RWKV_CMIX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}-TM_{RWKV_CMIX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 08:27:25,480] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
Traceback (most recent call last):
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/lightning_trainer.py", line 266, in <module>
    from src.model import RWKV
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/src/model.py", line 7, in <module>
    from .module.CoreDependencies import *
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/src/module/CoreDependencies.py", line 180, in <module>
    RWKV_CMIX_QVARS = os.environ.get("RWKV_CMIX_QVARS", "RKV").to_upper()
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'str' object has no attribute 'to_upper'. Did you mean: 'isupper'?


In [36]:
# Learning rate setting
LR_INIT="4e-3"
LR_FINAL="4e-4"

# Channel and timemix quantized settings
RWKV_CMIX_QTYPE="4bit"
RWKV_TIMX_QTYPE="4bit"

# Nuke python3 (for back to back run cleanup)
!killall -9 python3

# Run with the LRX setting
!cd "{TRAINER_DIR}" && \
    export RWKV_JIT_ON="0" && \
    export RWKV_TORCH_COMPILE="0" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    export RWKV_CMIX_QTYPE="{RWKV_CMIX_QTYPE}" && \
    export RWKV_TIMX_QTYPE="{RWKV_TIMX_QTYPE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_100k-world-32k-rechunk.yaml" \
        --model.load_model="../model/L{L_SIZE}-D{D_SIZE}-world-v6base-init.pth" \
        --model.lr_init="{LR_INIT}" \
        --model.lr_final="{LR_FINAL}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/v6-enwiki-100k-L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}-TM_{RWKV_CMIX_QTYPE}/" \
        --trainer.logger.init_args.name="{WANDB_PREFIX}L{L_SIZE}-D{D_SIZE}-CM_{RWKV_CMIX_QTYPE}-TM_{RWKV_CMIX_QTYPE} (Rechunk 4k, {DEEPSPEED_STAGE})" \
        --trainer.strategy="{DEEPSPEED_STAGE}" \
        --trainer.microbatch_size={BATCH_SIZE} \
        --trainer.devices="{GPU_DEVICES}"

python3: no process found


[2024-04-26 08:27:30,163] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-native' with torch '2.1.2'
Traceback (most recent call last):
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/lightning_trainer.py", line 266, in <module>
    from src.model import RWKV
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/src/model.py", line 7, in <module>
    from .module.CoreDependencies import *
  File "/home/recursal/RWKV-infctx-trainer/RWKV-v6-QT/src/module/CoreDependencies.py", line 180, in <module>
    RWKV_CMIX_QVARS = os.environ.get("RWKV_CMIX_QVARS", "RKV").to_upper()
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'str' object has no attribute 'to_upper'. Did you mean: 'isupper'?
